In [8]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import argrelextrema

In [9]:
#process data
#input: a file name
#output: number of AE events in that file, a list of record time, a list of AE events
def read_AE_file(filename):
    number_of_AE = 0
    record_time = []
    AE_list = []
    time_history = []
    with open(filename) as file:  
        #skip the header
        for _ in range(12):
            next(file)
        #read real values  
        for line in file:
            if "Record" in line: 
                time_history = []
                number_of_AE += 1
            elif "Timestamp" in line:
                record_time.append(float(line.split()[1]))
            elif line != '\n':
                time_history.append(float(line))  
                if  len(time_history) == 4000:
                    AE_list.append(time_history)
                
    return number_of_AE, record_time, AE_list

In [10]:
#calculate first arrival of a AE event
#input: time history
#output: first arrival time
def calculate_first_arrival(time_history):
    #use first 800 points to calculate noise level with mean and std
    event = np.array(time_history)
    noise = event[0:500]
    noise_mean = np.mean(noise)
    noise_std = np.std(noise)
    #set threshold
    upper_threshold = noise_mean + noise_std*50
    lower_threshold = noise_mean - noise_std*50
    #print(upper_threshold)
    #print(lower_threshold)
    
    #find local max and min index
    finding_region = event[500:2000]
    local_max_ind = argrelextrema(finding_region, np.greater)[0]
    local_min_ind = argrelextrema(finding_region, np.less)[0]
    local_ind = np.sort((np.concatenate((local_max_ind,local_min_ind))))
    #print(local_ind)
    
    #find first max or min
    max_min_index = 0
    for index in local_ind:
        if finding_region[index] > upper_threshold or finding_region[index] < lower_threshold:
            max_min_index = index+500
            break
    
    #print(max_min_index)
    #print(event[max_min_index])
    
    #elimate bad signals
    if max_min_index == 0:
        #print("bad signal")
        return max_min_index
    elif np.max(event) > 4 or np.min(event) < -4:
        #print("bad signal")
        return 0
    
    #find first_arrival
    else:
        for i in range(max_min_index-1,1,-1):            
            if event[i]*event[i-1] < 0:
                first_arrival = i
                break
            elif event[i] > event[i+1] and event[i] > event[i-1]:
                first_arrival = i
                break
            elif event[i] < event[i+1] and event[i] < event[i-1]:
                first_arrival = i
                break
    
    return first_arrival

In [11]:
AE_file_name = ["10_13_2021","10_25_2021","11_04_2021","11_12_2021","11_16_2021","11_30_2021"]
data_file_num = [["1","2","3","4"],
                 ["1","2","3","4","5","6","7","8","9","10"],
                 ["1","2","3","4","5","6","7","8","9","10","11"],
                 ["1","2","3","4","5","6","7","8"],
                 ["1","2","3","4","5","6","7","8","9"],
                 ["1","2","3","4","5","6","7","8"]
                ]
separation_record_time = [2781.22,1219.23,2323.43,2563.63,1455.7,2340.55]

In [12]:
AE_dataset = []
label = []
test_number = []
for i, file_name in enumerate(AE_file_name):
    print(i)
    file_num = data_file_num[i]
    for num in file_num:
        number_of_AE,record_time,AE_list = read_AE_file("C:/Users/huxxx/OneDrive/Desktop/SiC/SiC real specimen testing/"+ file_name + "/" + num + ".txt")
        for j in range(number_of_AE):
            first_arrival = calculate_first_arrival(AE_list[j])
            if first_arrival !=0:
                AE_dataset.append(AE_list[j][first_arrival:first_arrival+1000])
                if record_time[j] < separation_record_time[i]:
                    label.append(-1)
                else:
                    label.append(1)
                test_number.append(i)

0
1
2
3
4
5


In [23]:
AE_dataset = np.array(AE_dataset)
label = np.array(label)
test_number = np.array(test_number)
print(AE_dataset.shape)

print(np.count_nonzero(label == 1))
print(np.count_nonzero(label == -1))

(41029, 1000)
29975
11054


In [47]:
test_index = 5

training_index = []
testing_index = []
for i in range(AE_dataset.shape[0]-40):
    if label[i] == label[i+40]:
        if test_number[i] == test_index:
            testing_index.append(i)
        else:
            training_index.append(i)
            
print(len(training_index))
random_training_index = np.random.choice(training_index, 3000,replace=False)
print(random_training_index)

33693
[15258  4747 29943 ... 15519 28021 28182]


In [48]:
import matplotlib.pyplot as plt
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()

        self.conv1 = nn.Conv1d(1, 32, 7, stride=7)
        self.conv2 = nn.Conv1d(32, 64, 5, stride=5)
        self.conv3 = nn.Conv1d(64, 256, 3, stride=3)
        self.norm1 = nn.BatchNorm1d(32, affine=False)
        self.norm2 = nn.BatchNorm1d(64, affine=False)
        self.norm3 = nn.BatchNorm1d(256, affine=False)
        self.dropout1 = nn.Dropout(0.3)
        self.dropout2 = nn.Dropout(0.3)
        self.dropout3 = nn.Dropout(0.3)
        self.avepool = nn.AvgPool1d(40, stride=40)
        self.maxpool = nn.MaxPool1d(9, stride=8)
        self.fc1 = nn.Linear(256, 64)
        self.fc2 = nn.Linear(64, 16)
        self.fc3 = nn.Linear(16, 2)

    def forward(self, x):
        x = self.dropout1(F.relu(self.norm1(self.conv1(x))))
        x = self.dropout2(F.relu(self.norm2(self.conv2(x))))
        x = self.dropout3(F.relu(self.norm3(self.conv3(x))))
        x = (self.avepool(x.permute(2,1,0))).permute(2,1,0)
        x = self.maxpool(x)
        x = torch.flatten(x, 1) 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [49]:
model = Net()
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
mean = torch.zeros(40)
std = 0.5*torch.ones(40)
norm = transforms.Normalize((mean), (std))

X = torch.tensor(AE_dataset)
y = torch.tensor(label)
inputs = torch.reshape(X,(X.shape[0],1,X.shape[1]))
#change y to 0,1
labels = ((y+1)/2).long()

print(inputs.shape)
print(labels.shape)

torch.Size([41029, 1, 1000])
torch.Size([41029])


In [ ]:
loss_list = []

for epoch in range(1):
    for data_index in random_training_index:
        #print(data_index)
        batch_inputs = norm(inputs[data_index:data_index+40])
        batch_labels = torch.tensor([labels[data_index]])
        #print(batch_labels)
        #print(batch_inputs.shape)

        for t in range(200):
            # Forward pass: Compute predicted y by passing x to the model
            batch_outputs = model(batch_inputs.float())

            # Compute and print loss
            loss = criterion(batch_outputs, batch_labels)
            loss_list.append(loss)
            #if t % 100 == 99:
             # print(t, loss.item())

            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(data_index,loss.item())

plt.plot(loss_list)

15258 0.16045096516609192
4747 0.07078622281551361
29943 0.042383428663015366
32367 0.02711883746087551
14040 0.018315009772777557
8503 0.014648976735770702
15754 0.009762165136635303
17485 0.009950203821063042
22747 0.0076333265751600266
17741 0.005182285793125629
8631 0.004968439228832722
10786 0.27507346868515015
1180 0.1897277981042862
3256 0.19913245737552643
1190 0.20222067832946777
27857 0.17657701671123505
13100 0.055451326072216034
14150 0.19385233521461487
15092 0.0691336840391159
21698 0.17143617570400238
2163 0.16207292675971985
19888 0.051864393055438995
2844 0.028869090601801872
30786 0.022575052455067635
18727 0.010991856455802917
12787 0.19989077746868134
16730 0.15999972820281982
12427 0.13589632511138916
30360 0.14430351555347443
5487 0.05021790787577629
508 0.13878226280212402
33457 0.15734580159187317
4490 0.05141665041446686
13000 0.13311682641506195
26142 0.13356126844882965
6816 0.05588306114077568
29319 0.02723589539527893
29580 0.02237398736178875
19328 0.01145